# Meal picker

In [1]:
from common.my_settings import MySettings  
from common.utils import md
import dspy 
from typing import Literal

settings = MySettings().get()

Getting keys from environment variables


In [2]:
lm_gpt4omin = dspy.LM('gpt-4o-mini', temperature=0.9, model_type='chat', cache=False, api_key=settings.OPENAI_API_KEY)
dspy.configure(lm=lm_gpt4omin)

In [ ]:
class Meal(dspy.Signature):
    """A meal"""

    meal_suggestion: str = dspy.InputField(desc="A suggestion for a meal") 
    meal_exclusion: str = dspy.InputField(desc="A meal to avoid")           # Comment this out and see how the model repeats the meal unless asked not to

    name: str = dspy.OutputField(desc="Meal name")
    meal_type: Literal["Fish", "Red Meat", "Vegetarian"] = dspy.OutputField()

class MealEvaluator(dspy.Signature):
    """Is the meal a fish meal?"""

    meal: Meal = dspy.InputField(desc="A meal to evaluate")

    is_meal_type_fish: bool = dspy.OutputField(desc="Is the meal a fish meal?")

class MealDecider(dspy.Module):
    def __init__(self):
        self.num_attempts = 3
        self.choose_meal = dspy.ChainOfThought(Meal)
        self.evaluate_meal = dspy.Predict(MealEvaluator)

    def forward(self, meal_suggestion):
        previous_meal_name = ""
        for i in range(self.num_attempts):
            
            meal = self.choose_meal(
                meal_suggestion=meal_suggestion, 
                meal_exclusion=previous_meal_name
                )
            
            previous_meal_name = meal.name
            md('**Meal**: ', meal)
            result = self.evaluate_meal(meal=meal)
            md(result)

    
meal_decider = MealDecider()
meal_decider(meal_suggestion="Lunch")
        

**Meal**: Prediction(  
    reasoning="For lunch, a light yet satisfying meal is ideal. A Mediterranean quinoa salad with fresh vegetables and a drizzle of olive oil provides a healthy option that's filling without being heavy. It also aligns with the vegetarian category, making it suitable for a variety of dietary preferences.",  
    name='Mediterranean Quinoa Salad',  
    meal_type='Vegetarian'  
)

Prediction(  
    is_meal_type_fish=False  
)

**Meal**: Prediction(  
    reasoning='A suitable lunch option that avoids the Mediterranean Quinoa Salad could be a Grilled Salmon with a side of steamed broccoli. This meal provides healthy proteins and vegetables while respecting the exclusion.',  
    name='Grilled Salmon with Steamed Broccoli',  
    meal_type='Fish'  
)

Prediction(  
    is_meal_type_fish=True  
)

**Meal**: Prediction(  
    reasoning='Since grilled salmon is the meal to avoid, I suggest a vegetarian option that is both healthy and satisfying. A quinoa salad with mixed vegetables and a lemon vinaigrette would be a great alternative for lunch.',  
    name='Quinoa Salad with Mixed Vegetables',  
    meal_type='Vegetarian'  
)

Prediction(  
    is_meal_type_fish=False  
)

In [12]:
print(lm_gpt4omin.inspect_history(2))
print('################################################')
print(lm_gpt4omin.inspect_history())





[2025-09-12T18:06:39.406264]

System message:

Your input fields are:
1. `meal_suggestion` (str): A suggestion for a meal
2. `meal_exclusion` (str): A meal to avoid
Your output fields are:
1. `reasoning` (str): 
2. `name` (str): Meal name
3. `meal_type` (Literal['Fish', 'Red Meat', 'Vegetarian']):
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## meal_suggestion ## ]]
{meal_suggestion}

[[ ## meal_exclusion ## ]]
{meal_exclusion}

[[ ## reasoning ## ]]
{reasoning}

[[ ## name ## ]]
{name}

[[ ## meal_type ## ]]
{meal_type}        # note: the value you produce must exactly match (no extra characters) one of: Fish; Red Meat; Vegetarian

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        A meal


User message:

[[ ## meal_suggestion ## ]]
Lunch

[[ ## meal_exclusion ## ]]
Grilled Salmon with Steamed Broccoli

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`,